In [1]:
import pandas as pd

import camelot

import pdfplumber

import os

import re

import numpy as np

In [2]:
def process_all_pdfs_in_folder(source_path, sink_path, desired_disease):
    year_df = []
    for filename in os.listdir(source_path):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(source_path, filename)
            print(f"Processando o arquivo: {pdf_path}")
            week_df = extract_and_process_tables(pdf_path, desired_disease, filename)
            year_df.append(week_df)
            
    if year_df:
        concatenated_df = pd.concat(year_df, ignore_index=True)

        pdf_filename = source_path.rsplit('/', 1)[-1]
        print("Salvando")
        csv_filename = os.path.splitext(pdf_filename)[0] + desired_disease + '.csv'
        output_path = os.path.join(sink_path, csv_filename)
        concatenated_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Arquivo salvo: {csv_filename}")
            

In [3]:
def extract_and_process_tables(pdf_path, desired_disease, filename):
    all_dfs = []
    pdf = pdfplumber.open(pdf_path)

    for page_num in range(len(pdf.pages)):
        try:
            tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

            if tables:
                df = tables[0].df
                page = pdf.pages[page_num]
                table = page.extract_table()
                df_header = pd.DataFrame(table)
                                
                if (not df_header.empty 
                    and df_header.shape[0] > 1  # Verifica se há pelo menos 2 linhas
                    and df_header.shape[1] > 1  # Verifica se há pelo menos 2 colunas
                    and df_header.iloc[0, 1] is not None 
                    and isinstance(df_header.iloc[0, 1], str)
                    and desired_disease in df_header.iloc[0, 1]):  
                    
                    first = None
                    

                    for n in range(4, 10):
                        if df.iloc[n, 0]:
                            first = n
                            break
                            
                    df_all = process_table(df, df_header, filename, first)
                    
                        
                    if not df_all.empty:
                        all_dfs.append(df_all)
#                         print(all_dfs)
                        print(f"Processando a página {page_num + 1} com a tabela desejada.")

        except Exception as e:
            print(f"Erro ao processar a página {page_num + 1}: {e}")

    pdf.close()

    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
    else:
        final_df = pd.DataFrame()

    # Salva o DataFrame final em um arquivo CSV
#     pdf_filename = os.path.basename(pdf_path)
#     csv_filename = os.path.splitext(pdf_filename)[0] + '_filtered.csv'
#     output_path = os.path.join('/home/pirata/Documents/projects/epidemic_database/bases', csv_filename)
#     final_df.to_csv(output_path, index=False, encoding='utf-8-sig')

    return final_df

In [100]:
# Defina a função para processar as tabelas do PDF
def process_table(df, df_header, filename, first):
    
    
    if df.empty or df_header.empty:
        print("DataFrame ou cabeçalho vazio.")
        return pd.DataFrame()  # Retorna um DataFrame vazio se não houver dados
    
    weeknumber = filename[3:5]

    df = df.iloc[first:].reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df[:]

    # df.replace('', np.nan, inplace=True)

    df = df.loc[:, (df != '').any(axis=0)]

    df.dropna(axis=1, how='all')

    col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
    df.columns = col_names

    for col in col_names[1:len(df.columns)]:
        df[col] = df[col].str.replace(' ', '')

    first_col_name = df.columns[0]
    df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*|§FUENTE.*)')]


    indices_para_converter = range(1, len(df.columns))
    for idx in indices_para_converter:
        if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
            df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)

    num_col = len(df.columns)


    df1 = df.iloc[:, 0:5].copy()
    if len(df_header) > 1 and len(df_header.columns) > 1:
        df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
        df1['year'] = '2014'
    else:
        df1['disease'] = ''
        df1['year'] = ''


    df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)  

    df1['Acum_Year'] = '2013'
    df1['week'] = weeknumber
    df1['En_Estudio'] = ''

    novos_nomes = [
        'ENTIDAD FEDERATIVA',
        'Sem.',
        'M',
        'F',
        'Acum.'
    ]


    df1.columns = novos_nomes + list(df1.columns[5:])
    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
    df1 = df1.iloc[:, column_order]

    df2 = df.iloc[:, [0] + list(range(5, 10))].copy()

    # df2['Acum'] = ''

    if len(df_header) > 0 and len(df_header.columns) > 5:
        df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
        df2['year'] = '2014'
    else:
        df2['disease'] = ''
        df2['year'] = ''


    df2['MF'] = df2.get('Coluna8', 0) + df2.get('Coluna9', 0)

    df2['Acum_Year'] = '2013'

    df2['week'] = weeknumber

    column_order = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1]
    df2 = df2.iloc[:, column_order]


    df2.columns = novos_nomes + list(df2.columns[5:10]) + ['En_Estudio']
    column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
    df2 = df2.iloc[:, column_order]

    
    df_all = pd.concat([df1, df2], axis=0, ignore_index=True)
    

    return df_all

In [102]:

source_path = '/home/pirata/Documents/projects/epidemic_database/files/2014'
sink_path = '/home/pirata/Documents/projects/epidemic_database/bases/2014/teste'
desired_disease = 'Dengue'
process_all_pdfs_in_folder(source_path, sink_path, desired_disease)

Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2014/sem43.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2014/sem37.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2014/sem24.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-2 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-3 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-4 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-5 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-6 is image-based, camelot only works on tex

Processando a página 31 com a tabela desejada.


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-64 is image-based, camelot only works on text-based pages.
  warnings.warn(


Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2014/sem42.pdf


...

/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:365: UserWarning: No tables found in table area 2
  warnings.warn(f"No tables found in table area {table_idx + 1}")


Processando a página 29 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2014/sem31.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2014/sem33.pdf
Processando a página 31 com a tabela desejada.
Processando o arquivo: /home/pirata/Documents/projects/epidemic_database/files/2014/sem53.pdf


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-2 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-3 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-4 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-5 is image-based, camelot only works on text-based pages.
  warnings.warn(
/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-8 is image-based, camelot only works on tex

Processando a página 31 com a tabela desejada.
Salvando
Arquivo salvo: 2014Dengue.csv


/home/pirata/.local/lib/python3.8/site-packages/camelot/parsers/stream.py:448: UserWarning: page-64 is image-based, camelot only works on text-based pages.
  warnings.warn(


In [73]:
filename = 'sem01.pdf'

pdf_path = '/home/pirata/Documents/projects/epidemic_database/files/2014_teste/sem01.pdf'

page_num = 23

pdf = pdfplumber.open(pdf_path)

tables = camelot.read_pdf(pdf_path, pages=str(page_num + 1), flavor='stream')

if tables:
    dfx = tables[0].df
    page = pdf.pages[page_num]
    table = page.extract_table()
    df_header = pd.DataFrame(table)

In [74]:
first = None

for n in range(4, 10):
    if dfx.iloc[n, 0]:
        first = n
        break
        
        
print(n)

6


In [99]:
# weeknumber = re.search(r'-SE(\d+)\.pdf', filename).group(1)

weeknumber = filename[3:5]

df = dfx.iloc[6:].reset_index(drop=True)
df.columns = df.iloc[0]
df = df[:]

# df.replace('', np.nan, inplace=True)

df = df.loc[:, (df != '').any(axis=0)]

df.dropna(axis=1, how='all')

col_names = [f'Coluna{i+1}' for i in range(len(df.columns))]
df.columns = col_names

for col in col_names[1:len(df.columns)]:
    df[col] = df[col].str.replace(' ', '')

first_col_name = df.columns[0]
df = df[~df[first_col_name].str.match(r'^(TOTAL|FUENTE.*|§FUENTE.*)')]


indices_para_converter = range(1, len(df.columns))
for idx in indices_para_converter:
    if idx < len(df.columns):  # Verifica se o índice está dentro dos limites
        df.iloc[:, idx] = pd.to_numeric(df.iloc[:, idx], errors='coerce').fillna(0).astype(int)
        
num_col = len(df.columns)
        

df1 = df.iloc[:, 0:5].copy()
if len(df_header) > 1 and len(df_header.columns) > 1:
    df1['disease'] = df_header.iloc[0, 1].replace('\n', ' ')
    df1['year'] = '2014'
else:
    df1['disease'] = ''
    df1['year'] = ''
    
    
df1['MF'] = df1.get('Coluna3', 0) + df1.get('Coluna4', 0)  

df1['Acum_Year'] = '2013'
df1['week'] = weeknumber
df1['En_Estudio'] = ''

novos_nomes = [
    'ENTIDAD FEDERATIVA',
    'Sem.',
    'M',
    'F',
    'Acum.'
]


df1.columns = novos_nomes + list(df1.columns[5:])
column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
df1 = df1.iloc[:, column_order]

df2 = df.iloc[:, [0] + list(range(5, 10))].copy()

# df2['Acum'] = ''

if len(df_header) > 0 and len(df_header.columns) > 5:
    df2['disease'] = df_header.iloc[0, 5].replace('\n', ' ')
    df2['year'] = '2014'
else:
    df2['disease'] = ''
    df2['year'] = ''
    
    
df2['MF'] = df2.get('Coluna8', 0) + df2.get('Coluna9', 0)

df2['Acum_Year'] = '2013'

df2['week'] = weeknumber

column_order = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1]
df2 = df2.iloc[:, column_order]


df2.columns = novos_nomes + list(df2.columns[5:10]) + ['En_Estudio']
column_order = [0, 1, 2, 3, 7, 4, 8, 5, 6, 9, 10]
df2 = df2.iloc[:, column_order]




df2

,ENTIDAD FEDERATIVA,Sem.,M,F,MF,Acum.,Acum_Year,disease,year,week,En_Estudio
0,Aguascalientes,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,0
1,Baja California,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,0
2,Baja California Sur,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,1
3,Campeche,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,0
4,Coahuila,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,0
5,Colima,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,1
6,Chiapas,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,3
7,Chihuahua,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,0
8,Distrito Federal,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,0
9,Durango,0,0,0,0,0,2013,Fiebre Hemorrágica Por Dengue CIE-10ª REV. A91,2014,01,0


In [58]:
len(df.columns)

14

In [7]:
display(re.search(r'-SE(\d+)\.pdf', filename).group(1))

'51'

In [8]:
desired_disease = 'Dengue'

desired_disease in df_header.iloc[0, 1]

True

In [62]:
len(dfx.columns)

16

In [75]:
df_header

,0,1,2,3,4,5,6,7,8,9
0,ENTIDAD\nFEDERATIVA,Fiebre Por Dengue\nCIE-10ª REV.\nA90,None,None,None,Fiebre Hemorrágica Por Dengue\nCIE-10ª REV.\nA91,None,None,None,None
1,None,Confirmados,None,None,None,Confirmados,None,None,None,None
2,None,2014,None,None,2013,2014,None,None,None,2013
3,None,Sem.,Acum.,None,Acum.,En Estudio\nAcum.,Sem.,Acum.,None,None
4,None,None,M,F,None,None,None,M,F,Acum.
5,Aguascalientes -\nBaja California -\nBaja Cali...,None,- -\n- -\n- -\n- -,None,-\n-\n-\n-,-\n-\n1\n-,-\n-\n-\n-,- -\n- -\n- -\n- -,None,-\n-\n-\n-
6,,None,,None,,,,,None,
7,Coahuila -\nColima -\nChiapas -\nChihuahua -,None,- -\n- -\n- -\n- -,None,-\n-\n-\n-,-\n1\n3\n-,-\n-\n-\n-,- -\n- -\n- -\n- -,None,-\n-\n-\n-
8,,None,,None,,,,,None,
9,Distrito Federal -\nDurango -\nGuanajuato -\nG...,None,- -\n- -\n- -\n1 -,None,-\n-\n-\n-,-\n-\n-\n2,-\n-\n-\n-,- -\n- -\n- -\n- -,None,-\n-\n-\n-


In [76]:
dfx

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,FEDERATIVA,,,Confirmados,,,,,,,,Confirmados,,,,
1,,,2014,,,,2013,,,,2014,,,,,
2,,,,,,,,,,,,,,,,2013
3,,,,Acum.,,,,,En Estudio,,,,Acum.,,,
4,,Sem.,,,,,Acum.,,,Sem.,,,,,,
5,,,M,,F,,,,Acum.,,,M,,F,,Acum.
6,Aguascalientes,-,,-,,-,,-,-,,-,,-,,-,-
7,Baja California,-,,-,,-,,-,-,,-,,-,,-,-
8,Baja California Sur,-,,-,,-,,-,1,,-,,-,,-,-
9,Campeche,-,,-,,-,,-,-,,-,,-,,-,-
